## -=ВНИМАНИЕ!=-
Для корректной работы файла необходимо:
1. Папка с моделями должна находится на том же уровне, что и этот файл(если, конечно, оператор сам не поменяет расположение к ним)
2. Папка с изображениями должна находится на том же уровне, что и этот файл(если, конечно, оператор сам не поменяет расположение к ним)
3. Файл базы данных должен находится на том же уровне, что и этот файл(если, конечно, оператор сам не поменяет расположение к ним)   

Сабмит выгружается в папку: "output/submission"   
Изображения берутся из папки: "input/imgs_test"

## -=Подготовка к запуску=-

### Установка пакетов

In [142]:
%pip install ultralytics
%pip install numpy
%pip install opencv-python
%pip install easyocr
%pip install rapidfuzz
%pip install pandas

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 21.2.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy in c:\users\purp\appdata\local\programs\python\python310\lib\site-packages (1.25.2)



DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 21.2.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 21.2.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 21.2.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 21.2.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 21.2.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Импорт библиотек

In [143]:
import cv2
import numpy as np
import os
import pandas as pd
from rapidfuzz import process
from time import time
import matplotlib.pyplot as plt 
import csv

from ultralytics import YOLO
YOLO_VERBOSE=False

import easyocr

### Создание папок

In [144]:
images_dir = 'input/imgs_test'
csv_out_dir = "output/submission" 

directory_paths = [images_dir, csv_out_dir]

for dir in directory_paths:
    os.makedirs(dir, exist_ok=True)

## -=Загрузка моделей=-

Всего есть 3 модели:
- Распознаватель текста   
- Детектор артикула
- Детектор кода

In [145]:
reader = easyocr.Reader(lang_list=['ru'],
                        model_storage_directory="models",#'modules/models',
                        user_network_directory="models",#'modules/models',
                        recog_network='custom_ocr',
                        detector=False,
                        gpu=False,
                        )
detector_code = YOLO("models/yolo11_code_small.pt")
detector_art = YOLO("models/yolo11_art_small.pt")

Using CPU. Note: This module is much faster with a GPU.


{'ru'}
{'ru'}


## -=Загрузка базы=-

Выгружаем артикулы из базы, исключая дубликаты

In [146]:
db_path = "ДеталиПоПлануДляРазрешенныхЗаказов.xlsx"
df = pd.read_excel(db_path, engine='openpyxl')

articules = df["ДетальАртикул"].dropna().unique()

## -=Утилитарные функции=-

Общая функция для детекторов   
Если объект не найден, оставляет нулевой bbox, для плавности работы

In [147]:
def detect(type, image):
    if type == "code":
        results = detector_code(image, verbose=False)
    elif type == "art":
        results = detector_art(image, verbose=False)
    
    if len(results[0]) != 0:
        prediction = {
        'x': int(results[0].to("cpu").numpy().boxes.xywh[:, 0][0]),
        'y': int(results[0].to("cpu").numpy().boxes.xywh[:, 1][0]),
        'w': int(results[0].to("cpu").numpy().boxes.xywh[:, 2][0]),
        'h': int(results[0].to("cpu").numpy().boxes.xywh[:, 3][0]),
        }
        return prediction
    else:
        return {
            'x': 0,
            'y': 0,
            'w': 0,
            'h': 0,
        }

Функция для обрезки кода/артикула в целях распознавания

In [148]:
def crop_image(image_path, bbox):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    x, y, w, h = bbox
    cropped_part = image_rgb[y-h//2:y+h//2, x-w//2:x+w//2]
    return cropped_part

Функция для отрисовки bbox'ов в демонстративных целях

In [149]:
def draw_bbox_on_image(image, bbox):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    x, y, w, h = bbox
    cv2.rectangle(image_rgb, (x-w//2, y-h//2), (x+w//2, y+h//2), (255, 0, 0), 4)
    return image_rgb

Функция для распознавания символов на вырезанной части по bbox'у

In [150]:
def recognize_digits_ocr(image):
    ocr_result = reader.recognize(image)
    digits = list(str(ocr_result[0][-2]))

    return digits

Функция для вычисления среднего затраченного времени на операцию

In [151]:
def get_mean_time(start_time, end_time, num_iterations):
    total_time = end_time - start_time
    mean_time = total_time / num_iterations
    seconds = round(mean_time, 3)
    milliseconds = int((mean_time) * 1000)
    print(f"Среднее время обработки данного этапа {seconds} секунды или {milliseconds} миллисекунд")
    return mean_time

Функция для нормализации координат разметки bbox'ов YOLO, это необходимо, так как YOLO11 вовзращает не нормализованные координаты, а пиксельные

In [152]:
def convert_to_yolo(xmin, ymin, width, height, img_path):
    img = cv2.imread(img_path)
    img_height, img_width, _ = img.shape  # Height, Width, Channels

    center_x = (xmin + width / 2) / img_width
    center_y = (ymin + height / 2) / img_height

    norm_width = width / img_width
    norm_height = height / img_height

    return round(center_x, 6), round(center_y, 6), round(norm_width, 6), round(norm_height, 6)


## -=Загрузка путей к изображениям=-

Сохранение путей ко всем изображениям, для дальнейшего использования в детекторе и прочих модулях

In [153]:
files = os.listdir(images_dir)
image_files = [f for f in files if f.endswith(('.jpg', '.jpeg', '.png', '.JPG'))]
image_paths = [os.path.join(images_dir, f) for f in image_files]

## -=Поиск артикула и кода=-

Модуль, отвчающий за детекцию артикула и кода.   
1. Сначала изображение помещается в детектор артикула
2. Затем изображение помещается в детектор кода
3. После этого их данные собираются в список(и некоторая дополнительная информация впридачу) и отправляются дальше   
3.1. Если артикул или код не находится, то в далее вместо них отправляется пустая переменная и нулевой bbox.

In [154]:
cropped_data = []
start_time = time()
iterations = 0
for path in image_paths:
    iterations += 1
    art_detection_res = detect("art", path)
    if art_detection_res["x"] != 0 and art_detection_res["y"] != 0 and art_detection_res["w"] != 0 and art_detection_res["h"] != 0:
        image_art = crop_image(path, [art_detection_res["x"], art_detection_res["y"], art_detection_res["w"], art_detection_res["h"]])
        bbox_art = [art_detection_res["x"], art_detection_res["y"], art_detection_res["w"], art_detection_res["h"]]
    else:
        image_art = None
        bbox_art = [0,0,0,0]

    code_detection_res = detect("code", path)
    if code_detection_res["x"] != 0 and code_detection_res["y"] != 0 and code_detection_res["w"] != 0 and code_detection_res["h"] != 0:
        image_code = crop_image(path, [code_detection_res["x"], code_detection_res["y"], code_detection_res["w"], code_detection_res["h"]])
        bbox_code = [code_detection_res["x"], code_detection_res["y"], code_detection_res["w"], code_detection_res["h"]]
    else:
        image_code = None
        bbox_code = [0,0,0,0]

    cropped_data.append([image_art, image_code, path, bbox_art, bbox_code])
mean_time_detect = get_mean_time(start_time, time(),iterations)

Среднее время обработки данного этапа 0.138 секунды или 137 миллисекунд


## -=Распознавание артикула и кода=-

Модуль, отвчающий за распознавание артикула и кода.   
1. Данные последовательно помещаются в распознаватель
2. После этого результаты распознавания собираются в список(и старая дополнительная информация, которая пригодится позднее) и отправляются дальше   
2.1. Если артикул или код не распознаётся, то в далее вместо них отправляется пустая строка.

In [155]:
recog_results = []
start_time = time()
iterations = 1
for data in cropped_data:
    iterations += 1
    if data[0] is not None:
        art_recognition_res = recognize_digits_ocr(data[0])
    else:
        art_recognition_res = ""
    if data[1] is not None:
        code_recognition_res = recognize_digits_ocr(data[1])
    else:
        code_recognition_res = ""

    recog_results.append([art_recognition_res, code_recognition_res, data[2], data[3], data[4]])
mean_time_recog = get_mean_time(start_time, time(),iterations)

Среднее время обработки данного этапа 0.068 секунды или 68 миллисекунд


## -=Поиск по базе=-

Достаточно быстрый поиск построенный на использование высокоэффективной библиотеки RapidFuzz. Данная библиотека ищет схожие строки на ту, которая подаётся в запрос при помощи математической техники "Расстояния Левенштейна".   
В результате работы артикул либо исправляется на наиболее похожий, либо остаётся прежним, если он существует в базе "как есть".   
Код таким образом не ищется, так как некоторых кодов в базе нет и это испортит метрику, так как это может заменить валидные коды.

In [156]:
search_results = []
start_time = time()
iterations = 0
for recog_result in recog_results:
    iterations += 1
    if recog_result[0] != "":
        
        search_results_art = process.extractOne("".join(recog_result[0]), articules, score_cutoff=10)
    else:
        search_results_art = ""

    if search_results_art != "":
        search_results.append([search_results_art[0], recog_result[1], recog_result[2], recog_result[3], recog_result[4]])
    else:
        search_results.append([search_results_art, recog_result[1], recog_result[2], recog_result[3], recog_result[4]])

mean_time_base_search = get_mean_time(start_time, time(),iterations)

Среднее время обработки данного этапа 0.002 секунды или 2 миллисекунд


## -=Подсчет среднего времени=-

Модуль подсчета среднего времени обработки одного изображения.   
Собирается время со всех модулей, напрямую связанных с обработкой изображений, не считая загрузки данных и моделей, так как это происходит всего один раз за цикл работы приложения, когда запускается сервер.

In [157]:
all_mean_time = mean_time_detect + mean_time_recog + mean_time_base_search
seconds = round(all_mean_time, 3)
milliseconds = int((all_mean_time) * 1000)
print(f"Среднее время обработки всех этапов {seconds} секунды или {milliseconds} миллисекунд")

Среднее время обработки всех этапов 0.208 секунды или 207 миллисекунд


## -=Визуализация результатов=-   
В связи с уточнением критерия (картинок сразу нет, потому что файл с ними не влезает)

In [ ]:
for search_result in search_results:
    img_path = search_result[2]
    art = search_result[0].replace('"', '')
    code = search_result[1]
    final_string = f'"{art} {"".join(code)}"'
    image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    image_art = draw_bbox_on_image(image, search_result[3])
    image_art_code = draw_bbox_on_image(image_art, search_result[4])
    print("---------")
    print(os.path.basename(img_path))
    plt.imshow(image_art_code)
    plt.axis("off")
    plt.show()
    print(final_string)

## -=Выгрузка результатов=-

Создание датафрейма для выгрузки

In [159]:
# Список для хранения данных таблицы
data = []

# Обход папки с изображениями
for search_result in search_results:
    image_path = os.path.basename(search_result[2])

    yolo_bbox1 = convert_to_yolo(search_result[3][0], search_result[3][1], search_result[3][2], search_result[3][3],search_result[2])
    yolo_bbox2 = convert_to_yolo(search_result[4][0], search_result[4][1], search_result[4][2], search_result[4][3],search_result[2])

    bbox1 = f"0 {yolo_bbox1[0]} {yolo_bbox1[1]} {yolo_bbox1[2]} {yolo_bbox1[3]}"
    bbox2 = f"0 {yolo_bbox2[0]} {yolo_bbox2[1]} {yolo_bbox2[2]} {yolo_bbox2[3]}"
    label = f"{bbox1}\n{bbox2}"  

    art = search_result[0].replace('"', '')

    label_text = f'"{art} {"".join(search_result[1])}"'

    data.append([image_path, label, label_text])

df = pd.DataFrame(data, columns=['image_file', 'label', 'label_text'])



Выгрузка датафрейма в csv "submission"

In [160]:
df

,image_file,label,label_text
0,new100.JPG,0 0.0 0.0 0.0 0.0\n0 0.0 0.0 0.0 0.0,""" """
1,new101.JPG,0 0.678646 0.61875 0.355208 0.24375\n0 0.0 0.0...,"""195-30-1221 ТС1.1 """
2,new102.JPG,0 0.696875 0.596094 0.3625 0.30625\n0 0.0 0.0 ...,"""AM116.04.00.260 """
3,new103.JPG,0 0.585938 0.571094 0.338542 0.204687\n0 0.303...,"""195-30-1283 ТС1.1 9260"""
4,new104.jpg,0 0.0 0.0 0.0 0.0\n0 0.545693 0.629085 0.04306...,""" 30"""
...,...,...,...
108,new95.JPG,0 0.596354 0.588672 0.278125 0.202344\n0 0.642...,"""195-30-1061 ТС1.1 31610"""
109,new96.JPG,0 0.877604 0.40625 0.734375 0.1\n0 0.621354 0....,"""195-30-1285 ТС1.1 26599"""
110,new97.JPG,0 0.0 0.0 0.0 0.0\n0 0.439063 0.645312 0.17604...,""" 1661"""
111,new98.JPG,0 0.56875 0.604688 0.320833 0.184375\n0 0.6114...,"""АМ116.06.00.961 97"""


In [161]:
df.to_csv(csv_out_dir+'/submission.csv', index=False)